In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt

from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torch
from torch.autograd import Variable

In [ ]:
df = pd.read_csv("train.csv")
plt.imshow(np.reshape(df.values[0][1:],(28,28)), cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
train = pd.read_csv("train.csv")
train_labels = train['label'].values
train = train.drop("label",axis=1).values.reshape(len(train),1,28,28)

In [ ]:
X = torch.Tensor(train.astype(float))
y = torch.Tensor(train_labels).long()

In [ ]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 392)
        self.fc2 = nn.Linear(392, 196)
        self.fc3 = nn.Linear(196, 98)
        self.fc4 = nn.Linear(98, 10)
        
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [ ]:
model = MNISTClassifier()
loss_function = nn.NLLLoss()
opt = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(50):
    
    images = Variable(X)
    labels = Variable(y)

    opt.zero_grad()
    outputs = model(images)

    loss = loss_function(outputs, labels)
    loss.backward()
    opt.step()

    print ('Epoch [%d/%d] Loss: %.4f' %(epoch+1, 50, loss.data.item()))

In [ ]:
test = pd.read_csv("test.csv")
test_labels = test['label'].values
test = test.drop("label",axis=1).values.reshape(len(test),1,28,28)

X_test = torch.Tensor(test.astype(float))
y_test = torch.Tensor(test_labels).long()

In [ ]:
preds = model(X_test)

In [ ]:
print(preds[0])

In [ ]:
_, predictionlabel = torch.max(preds.data, 1)
predictionlabel = predictionlabel.tolist()

predictionlabel = pd.Series(predictionlabel)
test_labels = pd.Series(test_labels)

pred_table = pd.concat([predictionlabel, test_labels], axis=1)
pred_table.columns =['Predicted Value', 'True Value']

display(pred_table.head())

In [ ]:
preds = len(predictionlabel)
correct = len([1 for x,y in zip(predictionlabel, test_labels) if x==y])
print((correct/preds)*100)